# Project Description

Megaline is a mobile operator. Clients are offered two tariff plans: "Smart" and "Ultra". To adjust the advertising budget, the commercial department wants to understand which tariff brings in more money.
It is necessary to make a preliminary analysis of tariffs on a small sample of customers. The data of 500 Megaline users is available: who they are, where they are from, what tariff they use, how many calls and messages each sent in 2018. It is necessary to analyze the behavior of customers and draw a conclusion - which tariff is better.

# Plans Description

## "Smart" plan
- Monthly fee: 550 rubles
- Included 500 minutes of calls, 50 messages and 15 GB of internet traffic
- The cost of services above the tariff package:
- minute of conversation: 3 rubles
- message: 3 rubles
- 1 GB of Internet traffic: 200 rubles

## "Ultra" plan
- Monthly fee: 1950 rubles
- Included 3000 minutes of calls, 1000 messages and 30 GB of internet traffic
- The cost of services above the tariff package:
- minute of conversation: 1 ruble
- message: 1 ruble
- 1 GB of Internet traffic: 150 rubles

# Data Description

## Users table (user information):
- user_id - unique user ID
- first_name - username
- last_name - last name of the user
- age — user's age (years)
- reg_date — tariff connection date (day, month, year)
- churn_date — date when the tariff was discontinued (if the value is omitted, then the tariff was still valid at the time of data upload)
- city — user's city of residence
- tariff — tariff plan name

## Table calls (information about calls):
- id — unique call number
- call_date — call date
- duration — call duration in minutes
- user_id — identifier of the user who made the call

## Messages table (message information):
- id — unique message number
- message_date — message date
- user_id — identifier of the user who sent the message

## internet table (information about internet sessions):

- id — unique session number
- mb_used - the amount of Internet traffic spent per session (in megabytes)
- session_date — internet session date
- user_id - user ID

## Tariffs table (tariff information):
- tariff_name — tariff name
- rub_monthly_fee — monthly subscription fee in rubles
- minutes_included - the number of minutes of conversation per month included in the subscription fee
- messages_included - number of messages per month included in the subscription fee
- mb_per_month_included - the amount of Internet traffic included in the subscription fee (in megabytes)
- rub_per_minute - the cost of a minute of conversation in excess of the tariff package (for example, if the tariff includes 100 minutes of conversation per month, then a fee will be charged from 101 minutes)
- rub_per_message - the cost of sending a message in excess of the tariff package
- rub_per_gb - the cost of an additional gigabyte of Internet traffic in excess of the tariff package (1 gigabyte = 1024 megabytes)

# Part 1. Import data files, study general information

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
from IPython.display import display


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
# Let's define a function for printing a dataset and information about it

def print_df(table):
    
    display(table)
    print()
    table.info()

### Users

In [ ]:
#df_users = pd.read_csv('C:/Users/KDG/Google Drive/data science/4 Статистический анализ данных/project/users.csv', sep=',')
df_users = pd.read_csv('/datasets/users.csv', sep=',')
print_df(df_users)

### Calls

In [ ]:
df_calls = pd.read_csv('/datasets/calls.csv', sep=',')
print_df(df_calls)

### Messages

In [ ]:
df_messages = pd.read_csv('/datasets/messages.csv', sep=',')
print_df(df_messages)

### Internet

In [ ]:
df_internet = pd.read_csv('/datasets/internet.csv', sep=',')
print_df(df_internet)

### Тарифы

In [ ]:
df_tariffs = pd.read_csv('/datasets/tariffs.csv', sep=',')
print_df(df_tariffs)   

# Part 2. Data preparation

## Change the data types of the columns to more appropriate ones and fill in the gaps where needed

### Users

In [ ]:
# Let's define a function to print the unique values of each column. This is necessary in order to view the data "with the eyes"
def print_df_c(table):
    
    for name_col in table.columns:
        print(name_col)
        print()
        print(table[name_col].value_counts().sort_values(ascending=False).head(1000))
        print()
        print('---------------------------------------------------------------------------------------------------------------')
        print()

In [ ]:
print_df_c(df_users)

In [ ]:
df_users['churn_date'].fillna(0)
df_users['churn_date'] = pd.to_datetime(df_users['churn_date'], format='%Y-%m-%d')
df_users['reg_date'] = pd.to_datetime(df_users['reg_date'], format='%Y-%m-%d')
df_users.info()

In [ ]:
# Check for duplicates
df_users.duplicated().sum()

### Calls

In [ ]:
print_df_c(df_calls)

In [ ]:
# Lots of calls with zero duration. This indicates that the call was missed. It makes sense to remove these values,
# as further you will find the number of calls and the number of spent minutes by month

index = df_calls[df_calls['duration'] == 0].index
df_calls = df_calls.drop(index).reset_index(drop=True)


df_calls['call_date'] = pd.to_datetime(df_calls['call_date'], format='%Y-%m-%d')
df_calls['duration'] = df_calls['duration'].apply(np.ceil).astype('int64')
df_calls.info()

In [ ]:
# Check for duplicates
df_calls.duplicated().sum()

### Messages

In [ ]:
print_df_c(df_messages)

In [ ]:
df_messages['message_date'] = pd.to_datetime(df_messages['message_date'], format='%Y-%m-%d')
df_messages.info()

In [ ]:
# Check for duplicates
df_messages.duplicated().sum()

### Интернет

In [ ]:
print_df_c(df_internet)

In [ ]:
df_internet['session_date'] = pd.to_datetime(df_internet['session_date'], format='%Y-%m-%d')
df_internet['mb_used'] = df_internet['mb_used'].apply(np.ceil).astype('int64')
df_internet.info()

In [ ]:
# Check for duplicates
df_internet.duplicated().sum()

### Tariffs

In [ ]:
print_df_c(df_tariffs)
df_tariffs.info()

## Count data for each user

### Let's create a dataframe with all the statistics. Add the following monthly data to it:
- calls made
- spent minutes of conversation
- the number of messages sent
- the amount of Internet traffic used
- monthly revenue from each user

In [ ]:
# Let's add a column with operation months to each type of service
df_calls['month'] = df_calls['call_date'].dt.month
df_messages['month'] = df_messages['message_date'].dt.month
df_internet['month'] = df_internet['session_date'].dt.month

# Create dataframes with service costs for each type
count_calls = df_calls.groupby(['user_id', 'month']).agg({'user_id': 'count', 'duration': 'sum'}).rename(columns={'user_id':'count_calls'}).reset_index()
count_messages = df_messages.groupby(['user_id', 'month']).agg({'user_id': 'count'}).rename(columns={'user_id':'count_messages'}).reset_index()
count_internet = df_internet.groupby(['user_id', 'month']).agg({'mb_used': 'sum'}).reset_index()

print_df(count_calls)
print_df(count_messages)
print_df(count_internet)

In [ ]:
# Let's create a common dataframe with costs for each type of service

df_general = pd.merge(count_calls, count_messages, on=['user_id', 'month'], how='outer')
df_general = pd.merge(df_general, count_internet, on=['user_id', 'month'], how='outer')
df_general = pd.merge(df_general, df_users[['user_id', 'city', 'tariff']], on='user_id', how='outer')
df_tariffs = df_tariffs.rename(columns={'tariff_name':'tariff'})
df_general = pd.merge(df_general, df_tariffs, on='tariff', how='outer')


print_df(df_general)

In [ ]:
# There are two `users_id` with no indication of using the number. Let's see in more detail
print_df(df_users.query('user_id == "1128"'))
print_df(df_users.query('user_id == "1371"'))

In [ ]:
# They can be removed from the selection. They canceled the contract for communication a few days after the connection.
# They did not perform any actions, they can be removed from the selection
df_general = df_general.dropna(subset=['month']).reset_index(drop=True)
df_general.info()

In [ ]:
# Let's convert the "float" format to "int64" after replacing "NaN" with zeros in the number of messages and used Internet traffic
df_general['count_calls'] = df_general['count_calls'].fillna(0)
df_general['duration'] = df_general['duration'].fillna(0)
df_general['count_messages'] = df_general['count_messages'].fillna(0)
df_general['mb_used'] = df_general['mb_used'].fillna(0)

df_general['month'] = df_general['month'].astype('int64')
df_general['count_calls'] = df_general['count_calls'].astype('int64')
df_general['duration'] = df_general['duration'].astype('int64')
df_general['count_messages'] = df_general['count_messages'].astype('int64')
df_general['mb_used'] = df_general['mb_used'].astype('int64')


# Define a function for calculating revenue per user
def calc_profit(row):
    
    profit = row['rub_monthly_fee']
    
    add_messages = row['count_messages'] - row['messages_included']
    add_internet = (row['mb_used'] - row['mb_per_month_included']) / 1024
    
    if add_messages > 0: profit += add_messages * row['rub_per_message']
    if add_internet > 0: profit += add_internet * row['rub_per_gb']
    
    return profit

df_general['profit'] = df_general.apply(calc_profit, axis=1)
print_df(df_general)

# Part 3. Data analysis

## Calculate mean, variance and standard deviation, build histograms for:
- call duration
- number of sent messages
- used internet traffic

In [ ]:
# call duration in minutes
minutes_ultra_Moscow = df_general.loc[(df_general['tariff'] == 'ultra') & (df_general['city'] == 'Москва'), 'duration']
minutes_ultra_not_Moscow = df_general.loc[(df_general['tariff'] == 'ultra') & (df_general['city'] != 'Москва'), 'duration']
minutes_smart_Moscow = df_general.loc[(df_general['tariff'] == 'smart') & (df_general['city'] == 'Москва'), 'duration']
minutes_smart_not_Moscow = df_general.loc[(df_general['tariff'] == 'smart') & (df_general['city'] != 'Москва'), 'duration']


# number of sms
sms_ultra_Moscow = df_general.loc[(df_general['tariff'] == 'ultra') & (df_general['city'] == 'Москва'), 'count_messages']
sms_ultra_not_Moscow = df_general.loc[(df_general['tariff'] == 'ultra') & (df_general['city'] != 'Москва'), 'count_messages']
sms_smart_Moscow = df_general.loc[(df_general['tariff'] == 'smart') & (df_general['city'] == 'Москва'), 'count_messages']
sms_smart_not_Moscow = df_general.loc[(df_general['tariff'] == 'smart') & (df_general['city'] != 'Москва'), 'count_messages']


# amount of internet traffic
traf_ultra_Moscow = df_general.loc[(df_general['tariff'] == 'ultra') & (df_general['city'] == 'Москва'), 'mb_used']
traf_ultra_not_Moscow = df_general.loc[(df_general['tariff'] == 'ultra') & (df_general['city'] != 'Москва'), 'mb_used']
traf_smart_Moscow = df_general.loc[(df_general['tariff'] == 'smart') & (df_general['city'] == 'Москва'), 'mb_used']
traf_smart_not_Moscow = df_general.loc[(df_general['tariff'] == 'smart') & (df_general['city'] != 'Москва'), 'mb_used']

In [ ]:
# Get rid of zeros. In this case, the zeros distort the real picture.
def del_null(table):
    
    index = table.loc[table == 0].index
    return table.drop(index).reset_index(drop=True)


minutes_ultra_Moscow = del_null(minutes_ultra_Moscow)
minutes_ultra_not_Moscow = del_null(minutes_ultra_not_Moscow)
minutes_smart_Moscow = del_null(minutes_smart_Moscow)
minutes_smart_not_Moscow = del_null(minutes_smart_not_Moscow)

sms_ultra_Moscow = del_null(sms_ultra_Moscow)
sms_ultra_not_Moscow = del_null(sms_ultra_not_Moscow)
sms_smart_Moscow = del_null(sms_smart_Moscow)
sms_smart_not_Moscow = del_null(sms_smart_not_Moscow)

traf_ultra_Moscow = del_null(traf_ultra_Moscow)
traf_ultra_not_Moscow = del_null(traf_ultra_not_Moscow)
traf_smart_Moscow = del_null(traf_smart_Moscow)
traf_smart_not_Moscow = del_null(traf_smart_not_Moscow)

In [ ]:
# определим функции для нахождения верхних и нижних границ
def bot_line(table):
    
    Q1 = table.quantile(0.25)
    Q3 = table.quantile(0.75)
    IQR = Q3 - Q1
    return Q1 - 1.5*IQR
    
    
def top_line(table):
    
    Q1 = table.quantile(0.25)
    Q3 = table.quantile(0.75)
    IQR = Q3 - Q1
    return Q3 + 1.5*IQR    

# function for plotting a histogram and a box with a mustache
def hist_box_do(table, table_name):
    
    mean_table = table.mean()
    median_table = table.median()
    variance_table = np.var(table, ddof=1)   
    standart_dev_table = np.std(table, ddof=1)  
    
    bot = bot_line(table)
    top = top_line(table)
    
    plt.hist(table)
    plt.title(table_name)
    plt.show()
    
    plt.boxplot(table)
    plt.show()
    
    print('Q1-1.5IQR =', bot)
    print('Q3+1.5IQR =', top)
    print('Mean:', mean_table)
    print('Median:', median_table)
    print('Dispersion:', variance_table) 
    print('Standard deviation:', standart_dev_table)
    print('-------------------------------------------------------------------------------------------------------------------')

In [ ]:
# Let's build graphs
table_names = ['minutes_ultra_Moscow', 'minutes_ultra_not_Moscow', 'minutes_smart_Moscow', 'minutes_smart_not_Moscow',
               'sms_ultra_Moscow', 'sms_ultra_not_Moscow', 'sms_smart_Moscow', 'sms_smart_not_Moscow',
               'traf_ultra_Moscow', 'traf_ultra_not_Moscow', 'traf_smart_Moscow', 'traf_smart_not_Moscow'                 
              ]  

hist_box_do(minutes_ultra_Moscow, table_names[0])
hist_box_do(minutes_ultra_not_Moscow, table_names[1])
hist_box_do(minutes_smart_Moscow, table_names[2])
hist_box_do(minutes_smart_not_Moscow, table_names[3])

hist_box_do(sms_ultra_Moscow, table_names[4])
hist_box_do(sms_ultra_not_Moscow, table_names[5])
hist_box_do(sms_smart_Moscow, table_names[6])
hist_box_do(sms_smart_not_Moscow, table_names[7])

hist_box_do(traf_ultra_Moscow, table_names[8])
hist_box_do(traf_ultra_not_Moscow, table_names[9])
hist_box_do(traf_smart_Moscow, table_names[10])
hist_box_do(traf_smart_not_Moscow, table_names[11])

### Remove outliers

In [ ]:
def blowout(table): 
    
    bot = bot_line(table)
    top = top_line(table)
    index = table.loc[(table < bot) | (table > top)].index
    table = table.drop(index).reset_index(drop=True)
    return table

minutes_ultra_Moscow = blowout(minutes_ultra_Moscow)
minutes_ultra_not_Moscow = blowout(minutes_ultra_not_Moscow)
minutes_smart_Moscow = blowout(minutes_smart_Moscow)
minutes_smart_not_Moscow = blowout(minutes_smart_not_Moscow)

sms_ultra_Moscow = blowout(sms_ultra_Moscow)
sms_ultra_not_Moscow = blowout(sms_ultra_not_Moscow)
sms_smart_Moscow = blowout(sms_smart_Moscow)
sms_smart_not_Moscow = blowout(sms_smart_not_Moscow)

traf_ultra_Moscow = blowout(traf_ultra_Moscow)
traf_ultra_not_Moscow = blowout(traf_ultra_not_Moscow)
traf_smart_Moscow = blowout(traf_smart_Moscow)
traf_smart_not_Moscow = blowout(traf_smart_not_Moscow)

In [ ]:
hist_box_do(minutes_ultra_Moscow, table_names[0])
hist_box_do(minutes_ultra_not_Moscow, table_names[1])
hist_box_do(minutes_smart_Moscow, table_names[2])
hist_box_do(minutes_smart_not_Moscow, table_names[3])

hist_box_do(sms_ultra_Moscow, table_names[4])
hist_box_do(sms_ultra_not_Moscow, table_names[5])
hist_box_do(sms_smart_Moscow, table_names[6])
hist_box_do(sms_smart_not_Moscow, table_names[7])

hist_box_do(traf_ultra_Moscow, table_names[8])
hist_box_do(traf_ultra_not_Moscow, table_names[9])
hist_box_do(traf_smart_Moscow, table_names[10])
hist_box_do(traf_smart_not_Moscow, table_names[11])

## Conclusion

Based on the histograms, we can make an obvious conclusion that SMS is rarely used by anyone.

In the tariff `"Ultra"`: <br />
Both in Moscow and in the regions, users do not even go beyond the limits of the service package. <br />
In the tariff `"Smart"`: <br />
Both in Moscow and in the regions, users are close to going beyond the limits for the number of minutes and SMS, and slightly go beyond the limits of the standard Internet traffic package.

# Part 4. Hypothesis testing

### Null hypothesis:
average revenue from users of Ultra tariffs is equal to average revenue from users of Smart tariffs <br />
### Alternative:
average revenue from Ultra users is NOT equal to average revenue from Smart users <br />
### alpha:
0.05

### Null hypothesis:
average revenue from users of Ultra tariffs is equal to average revenue from users of Smart tariffs <br />
### Alternative:
average revenue from Ultra users is NOT equal to average revenue from Smart users <br />
### alpha:
0.05

In [ ]:
profit_tariff_ultra = df_general.loc[df_general['tariff'] == 'ultra', 'profit']
profit_tariff_smart = df_general.loc[df_general['tariff'] == 'smart', 'profit']

alpha = 0.05

results = st.ttest_ind(
    profit_tariff_ultra, 
    profit_tariff_smart)

print('p-value: ', results.pvalue)

if (results.pvalue < alpha):
    print("Rejecting the null hypothesis")
else:
    print("Failed to reject the null hypothesis")

## Conclusion
The null hypothesis was rejected, which means that the average revenue from Ultra tariff users is not equal to the average revenue from Smart tariff users.

## Testing the hypothesis that the average revenue from users from Moscow differs from revenue from users from other regions

### Null hypothesis:
average revenue from users from Moscow is equal to revenue from users from other regions <br />
### Alternative:
average revenue from users from Moscow is NOT equal to revenue from users from other regions <br />
### alpha:
0.05

In [ ]:
profit_p_from_Moscow = df_general.loc[df_general['city'] == 'Москва', 'profit']
profit_p_not_from_Moscow = df_general.loc[df_general['city'] != 'Москва', 'profit']

alpha = 0.05

results = st.ttest_ind(
    profit_p_from_Moscow, 
    profit_p_not_from_Moscow)

print('p-value: ', results.pvalue)

if (results.pvalue < alpha):
    print("Rejecting the null hypothesis")
else:
    print("Failed to reject the null hypothesis")

## Conclusion
The null hypothesis was not rejected, which means that there are no significant differences between the samples.

# Part 5. Conclusion

### Total Moscow revenue for the year for each tariff, rub:

In [ ]:
profit_p_from_Moscow_ultra = df_general.loc[(df_general['tariff'] == 'ultra') & (df_general['city'] == 'Москва'), 'profit']
profit_p_from_Moscow_smart = df_general.loc[(df_general['tariff'] == 'smart') & (df_general['city'] == 'Москва'), 'profit']

print('Revenue')
print('Ultra', profit_p_from_Moscow_ultra.sum())
print('Smart', profit_p_from_Moscow_smart.sum())
print()

print('Call duration')
print('Ultra', minutes_ultra_Moscow.mean())
print('Smart', minutes_smart_Moscow.mean())
print()

print('Number of SMS')
print('Ultra', sms_ultra_Moscow.mean())
print('Smart', sms_smart_Moscow.mean())
print()

print('Internet traffic')
print('Ultra', traf_ultra_Moscow.mean())
print('Smart', traf_smart_Moscow.mean())
print()

### Total regional revenue for the year for each tariff, rub:

In [ ]:
profit_p_not_from_Moscow_ultra = df_general.loc[(df_general['tariff'] == 'ultra') & (df_general['city'] != 'Москва'), 'profit']
profit_p_not_from_Moscow_smart = df_general.loc[(df_general['tariff'] == 'smart') & (df_general['city'] != 'Москва'), 'profit']

print('Revenue')
print('Ultra', profit_p_not_from_Moscow_ultra.sum())
print('Smart', profit_p_not_from_Moscow_smart.sum())
print()

print('Call duration')
print('Ultra', minutes_ultra_not_Moscow.mean())
print('Smart', minutes_smart_not_Moscow.mean())
print()

print('Number of SMS')
print('Ultra', sms_ultra_not_Moscow.mean())
print('Smart', sms_smart_not_Moscow.mean())
print()

print('Internet traffic')
print('Ultra', traf_ultra_not_Moscow.mean())
print('Smart', traf_smart_not_Moscow.mean())
print()

## Conclusions:
1. The average duration of calls, number of SMS and Internet traffic is close to the "Smart" tariff both in the regions and in Moscow.
2. The revenue from the "Ultra" tariff in Moscow is slightly higher than from the "Smart" tariff, but in the region, the "Smart" tariff is confidently leading in terms of revenue.

### Therefore, from the point of view of revenue, in Moscow it will be more profitable to advertise the "Ultra" tariff, and in the regions the "Smart" tariff.

### Average user spending by tariffs
Tariff `"Ultra"`: <br />

Both in Moscow and in the regions, users do not even go beyond the limits of the service package.

Tariff `"Smart"`: <br />

Both in Moscow and in the regions, users are close to going beyond the limits for the number of minutes and SMS, and slightly go beyond the limits of the standard Internet traffic package.